# Rapport Projet : Prédiction de l'Hypertension

**Nom & Prenoms :** ZINTA ARTHUR ANGE EEMANUEL  
**Date :** 04/12/2025  
**Cours / Master :** Master Data Science


## Introduction

L’hypertension artérielle constitue un facteur majeur de risque cardiovasculaire.  
Dans une démarche de prévention, ce projet vise à exploiter des données cliniques afin de prédire la probabilité qu’un patient développe une hypertension.

Pour ce faire, j’ai réalisé :
- une analyse exploratoire des données,
- un prétraitement complet (nettoyage, standardisation, gestion des outliers),
- puis l’entraînement de plusieurs modèles de Machine Learning : Logistic Regression, Random Forest et XGBoost.

L’objectif final est d’identifier précocement les patients à risque et de proposer des mesures de prévention adaptées.


##  Description des données

Le jeu de données utilisé contient des informations cliniques sur un ensemble de patients.  
Ces variables permettent d’évaluer l’état de santé général et de détecter les facteurs pouvant influencer le risque d’hypertension.

### Composition du dataset
- **Nombre d’observations :** 70.000
- **Nombre de variables :** 13
- **Variable cible :** `cardio`
  - 0 = Patient non hypertendu  
  - 1 = Patient hypertendu  

### Types de variables

- continue : age , weight,height , ap_hi , ap_lo
- categorielle : gender , gluc , cholesterol , smoke , alco , active , cardio
Le tableau suivant montre un aperçu des premières lignes du dataset :


|    | id | age | gender | height | weight | ap_hi | ap_lo | cholesterol | gluc | smoke | alco | active | cardio |
|----|----|-----|--------|--------|--------|-------|-------|-------------|------|-------|------|--------|--------|
| 0  | 0  | 50  | 2      | 168    | 62.0   | 110   | 80    | 1           | 1    | 0     | 0    | 1      | 0      |
| 1  | 1  | 55  | 1      | 156    | 85.0   | 140   | 90    | 3           | 1    | 0     | 0    | 1      | 1      |
| 2  | 2  | 52  | 1      | 165    | 64.0   | 130   | 70    | 3           | 1    | 0     | 0    | 0      | 1      |
| 3  | 3  | 48  | 2      | 169    | 82.0   | 150   | 100   | 1           | 1    | 0     | 0    | 1      | 1      |
| 4  | 4  | 48  | 1      | 156    | 56.0   | 100   | 60    | 1           | 1    | 0     | 0    | 0      | 0      |


### VISUALISATION DE LA DISTRIBUTION DE LA TARGET

```python
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
bins = [-0.5, 0.5, 1.5]

plt.hist(df['cardio'], bins=bins, color='skyblue', edgecolor='black', rwidth=0.8)
plt.xticks([0, 1])  # s'assurer que seuls 0 et 1 apparaissent sur l'axe X
plt.title("Distribution de la variable cible")
plt.xlabel("Cardio (0 = non hypertendu, 1 = hypertendu)")
plt.ylabel("Nombre de patients")
plt.show()
()

```

![diribution_target](distribution_target.png)

### Interpretation

Il y a autant de patients atteints de maladies cardiovasculaires que de patients non atteints. 

## Prétraitement 

Le prétraitement a permis de nettoyer, normaliser et encoder les données pour les rendre exploitables par le modèle. Le dataset a ensuite été divisé en ensembles d’entraînement, de validation et de test, assurant un apprentissage efficace, un réglage optimal des hyperparamètres et une évaluation objective des performances.

```python

# Créer la colonne BMI
df["BMI"] = df["weight"] / (df["height"]/100)**2

# Filtrer les valeurs aberrantes (optionnel mais recommandé)
df = df[(df["ap_hi"] >= 80) & (df["ap_hi"] <= 240)]
df = df[(df["ap_lo"] >= 50) & (df["ap_lo"] <= 150)]
df = df[(df["weight"] >= 30) & (df["weight"] <= 200)]
df = df[(df["height"] >= 140) & (df["height"] <= 210)]

# Séparer features et target
X = df.drop(columns=[ "cardio"])
y = df["cardio"]

# Split train/test/validation
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Standardisation des features numériques
num_features = ["age", "height", "weight", "ap_hi", "ap_lo", "BMI"]
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_val[num_features] = scaler.transform(X_val[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

```

## Modelling

- Création de modèle machine learning pour entrainer nos données

```python

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Fonction d'évaluation
def evaluate_model(model, X_val, y_val, X_test, y_test):
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    print(f"Performance sur le val set :")
    print("Accuracy :", accuracy_score(y_val, y_val_pred))
    print("F1-score :", f1_score(y_val, y_val_pred))
    print("\nPerformance finale sur le test set :")
    print("Accuracy :", accuracy_score(y_test, y_test_pred))
    print("F1-score :", f1_score(y_test, y_test_pred))
    print("\nClassification report :\n", classification_report(y_test, y_test_pred))

# Modèles
models = {
    "LogisticRegression": LogisticRegression(solver='liblinear', random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Entraînement et évaluation
for name, model in models.items():
    print(f"--- Évaluation du modèle : {name} ---")
    model.fit(X_train, y_train)
    evaluate_model(model, X_val, y_val, X_test, y_test)

```

### Interpreation

Après avoir entraîné plusieurs modèles et évalué leurs performances à l’aide de notre fonction d’évaluation, il ressort que **XGBoost** est le modèle le plus performant pour la prédiction.  
Il obtient une **accuracy de 0.73 sur le jeu de test**, ce qui en fait le meilleur choix parmi les modèles testés.


### Optimisation des hyperparamètres du modèle choisi avec GridSearchCV

Nous optimisons les hyperparamètres du modèle en utilisant GridSearchCV.
Cette méthode teste toutes les combinaisons possibles d’hyperparamètres définies dans une grille afin de trouver celle qui donne les meilleures performances sur le jeu de validation.
L’optimisation permet au modèle d’apprendre de manière plus efficace et d’améliorer sa précision sur les données.


```python

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

best_xgb = grid_search.best_estimator_
print("Meilleurs paramètres :", grid_search.best_params_)
evaluate_model(best_xgb, X_val, y_val, X_test, y_test)
```

### Test
Après avoir entraîné plusieurs modèles et évalué leurs performances avec notre fonction d’évaluation, il ressort que **XGBoost est le modèle le plus performant**, atteignant une accuracy de **0,73** sur le jeu de test.


```python

import pandas as pd

# Ordre des features utilisé dans ton modèle (doit correspondre à X_train)
feature_order = ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 
                 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'BMI']
scaler_features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'BMI']

# Profils patients
patient_healthy = {
    'age': 70,
    'gender': 1,
    'height': 170,
    'weight': 75,
    'ap_hi': 120,
    'ap_lo': 80,
    'cholesterol': 1,
    'gluc': 1,
    'smoke': 0,
    'alco': 0,
    'active': 1,
    'BMI': 75 / (1.70**2)
}

patient_risk = {
    'age': 70,
    'gender': 1,
    'height': 170,
    'weight': 120,
    'ap_hi': 180,
    'ap_lo': 110,
    'cholesterol': 3,
    'gluc': 3,
    'smoke': 1,
    'alco': 1,
    'active': 0,
    'BMI': 120 / (1.70**2)
}

# Créer DataFrame avec le bon ordre
df_patients = pd.DataFrame([
    [patient_healthy[col] for col in feature_order],
    [patient_risk[col] for col in feature_order]
], columns=feature_order)

# Standardiser seulement les colonnes numériques
df_patients[scaler_features] = scaler.transform(df_patients[scaler_features])

# Prédictions
pred_class = best_xgb.predict(df_patients)
# Probabilités prédites pour nos patients
pred_proba = best_xgb.predict_proba(df_patients)

# Choisir un seuil personnalisé, par exemple 0.65
threshold = 0.65

# Classe prédite selon le seuil
pred_class_custom = (pred_proba[:, 1] >= threshold).astype(int)

# Affichage
for i, patient_name in enumerate(['Sain', 'À risque']):
    print(f"\nProfil {patient_name} :")
    print("Classe prédite (seuil 0.65) :", pred_class_custom[i])
    print("Probabilité d'hypertension :", pred_proba[i])


```

### Resultat

| Profil            | Classe prédite (seuil = 0.65) | P(0)          | P(1)          |
|-------------------|-------------------------------|---------------|---------------|
| **Profil Sain**   | 0                             | 0.3830126     | **0.6169874** |
| **Profil À risque** | 1                           | 0.15896243    | **0.8410376** |



##  Learning curve

## Learning Curve

La *learning curve* est un outil d’évaluation qui permet d’analyser la capacité d’un modèle à apprendre progressivement.  
Elle montre comment évoluent les performances du modèle sur :

- **les données d’entraînement**,  
- **les données de validation**,  

au fur et à mesure que la quantité de données utilisées pour l’apprentissage augmente.

Cette courbe met également en évidence :
- **la moyenne des scores** obtenus,
- **l’écart-type**, qui indique la stabilité ou la variabilité des performances.

Elle permet ainsi d’identifier si le modèle souffre :
- **d’overfitting** (trop bon sur train, mauvais sur validation),
- **d’underfitting** (mauvais sur les deux),
- ou s’il généralise correctement.


```python
from sklearn.model_selection import learning_curve
import numpy as np
train_size , train_score, test_score = learning_curve (

    estimator=best_xgb,
    X= X_train,
    y= y_train,
    cv=5,
    scoring="accuracy",
    train_sizes=np.linspace(0.1,1.0,10),
    n_jobs=1
)

```

![learning_curve](learning_curve.png)

###  Interprétation

Sur la courbe d’apprentissage, le score d’accuracy finale obtenu sur les **données d’entraînement** est de **0.752**, tandis que celui obtenu sur les **données de validation** est de **0.735**.  

L’écart entre les deux scores est faible, ce qui indique **l’absence de surapprentissage (overfitting)**.  
Ainsi, le modèle parvient à généraliser correctement : ses performances en validation sont proches de celles observées à l’entraînement, ce qui témoigne d’une **prédiction fiable et stable**.


## Prise de décision 

Après avoir mener ce travail sur les données cliniques de different patient on envisage les decisions suivantes :


| Probabilité prédite | Décision médicale           | Explication                                                                 |
|-------------------|----------------------------|---------------------------------------------------------------------------|
| **P < 0.30**       | Patient sain               | Aucun signe d’hypertension, simple suivi normal                           |
| 0.30 ≤ P < 0.65    | Surveiller (risque modéré) | Recommandations : perte de poids, activité, vérifier la tension chaque mois |
| **P ≥ 0.65**       | Patient à haut risque      | Doit être orienté pour dépistage + mesures préventives                     |


## Conclusion

Ce projet a permis de développer un modèle de prédiction de l'hypertension basé sur des données cliniques, avec XGBoost un modèle de machine learning qui offre la meilleure performance. Les résultats montrent que notre approche peut aider à identifier les patients à risque et soutenir les actions de prévention.

À l'avenir, ce modèle pourrait être enrichi avec des données supplémentaires, comme des habitudes alimentaires ou des mesures génétiques, afin d'améliorer encore la précision et de contribuer à une prévention plus personnalisée des maladies cardiovasculaires.
